In [1]:
from main import main as execute

import os
import pandas as pd
from typing import List, Dict, Tuple

In [2]:
################################################
# INPUT CONFIGURATION- REPLACES OLD .JSON FILE #
################################################

input = {
    "DEVICE_NAME": "CAM3",
    "DEVICE_TYPE": "CAMERA",
    
    "DEVICE_SPECIES": "DIGICAM",
    
    "EXP_SHOT_NOS": [3],
    "BKG_SHOT_NOS": [1],
    
    "BKG_NAME": "DARKFIELD",
    "BACKGROUND_STATUS": "RAW",
    
    "OPERATIONS": {
        
        "PLOT": True,
        
        "AVERAGE_SHOTS":False
        
    }
}

In [3]:
# Parent data path
data_path = '/eos/project/h/hiradmat/HRMT Experiments/2024/FIREBALL2/FB2_HRMT64_repository/HRMT64 data'

# Check to make sure that the parent data path actually exists.
if not os.path.exists(data_path):
    raise NotImplementedError("Error: parent data_path doesn't exist.")

In [10]:
# Path to shot log
shotlog_path = "./example_data/shotlog.csv"

if not os.path.exists(shotlog_path):
    raise NotImplementedError("Error: path to shot log not found.")

In [5]:
# List of subdirectories in parent data path which should
folders = ["SCOPE1", "SCOPE2", "EP_Spectrometer_Cams", "PlasmaCell_Cams"]

In [6]:
# Make sure all of the folders actually exist.
for folder in folders:
    new_path = os.path.join(data_path, folder)
    if not os.path.exists(new_path):
        raise NotImplementedError(f"Path {new_path} to folder {folder} does not exist.\n")

In [ ]:
# Template file names at which the data will be stored

template_fnames = {
    "DIGICAM": 'HRM{}.DigiCam_OD{}_{}_{}.csv',
    "PROBE": 'C{}_SCOPE{}_{}.csv'
}